# Custom Gates and Decomposition in Cirq

This notebook shows how to define custom gates to encapsulate multi-step patterns, how Cirq decomposes them, and how to selectively keep or expand custom structure.

## 1. Imports

In [ ]:
import cirq
from typing import Sequence

## 2. Defining Custom Gates
We'll create gates for Bell pair preparation and message preparation used in teleportation.

In [ ]:
class BellPairGate(cirq.Gate):
    def _num_qubits_(self):
        return 2
    def _decompose_(self, qubits):
        c, t = qubits
        yield cirq.H(c)
        yield cirq.CNOT(c, t)
    def _circuit_diagram_info_(self, args):
        return 'BP','BP'

class MessagePrepGate(cirq.Gate):
    def _num_qubits_(self):
        return 2
    def _decompose_(self, qubits):
        m, a = qubits
        yield cirq.CNOT(m, a)
        yield cirq.H(m)
    def _circuit_diagram_info_(self, args):
        return 'PM_M','PM_A'

BP = BellPairGate()
MP = MessagePrepGate()

## 3. Teleportation with Custom Gates

In [ ]:
def simple_teleportation() -> Sequence[cirq.Operation]:
    alice = cirq.NamedQubit('alice')
    bob = cirq.NamedQubit('bob')
    msg = cirq.NamedQubit('msg')

    program: Sequence[cirq.Operation] = []
    # 1. Prepare Bell pair between alice and bob
    program.append(BP.on(alice, bob))
    # 2. Prepare message qubit in Rx(0.7)|0>
    program.append(cirq.rx(0.7)(msg))
    # 3. Entangle message with alice (Bell + measurement portion of teleportation)
    program.append(MP.on(msg, alice))
    # 4. Measure message and alice to obtain classical bits b1, b2
    program.append(cirq.measure(msg, key='b1'))
    program.append(cirq.measure(alice, key='b2'))
    # 5. Apply feed-forward corrections to bob
    program.append(cirq.Z.on(bob).with_classical_controls('b1'))
    program.append(cirq.X.on(bob).with_classical_controls('b2'))
    program.append(cirq.measure(bob, key='result'))
    return program

## 4. Inspect the Circuit with Custom Gates

In [ ]:
circuit = cirq.Circuit(simple_teleportation())
print('Circuit with custom gates:')
print(circuit)

## 5. Full Decomposition
We can ask Cirq to decompose all gates to primitive operations.

In [ ]:
decomposed_full = cirq.Circuit(cirq.decompose(circuit))
print('Fully decomposed:')
print(decomposed_full)

## 6. Selective Decomposition

The `keep` predicate controls where recursive decomposition stops:
- If `keep(op)` returns True, the operation is left as-is.
- If it returns False, Cirq tries to expand the operation via its decomposition methods.

In the example below, we decompose only the custom gates (`BellPairGate`, `MessagePrepGate`) by returning False for them and True for everything else.

In [ ]:
def keep(op: cirq.Operation) -> bool:
    gate = getattr(op, 'gate', None)
    return not isinstance(gate, (BellPairGate, MessagePrepGate))

decomposed_custom = cirq.Circuit(cirq.decompose(circuit, keep=keep))
print('Decomposed custom gates removed:')
print(decomposed_custom)

## 7. Simulation

We run the teleportation circuit several times to confirm Bob receives (approximately) the same state that was prepared on the message qubit. Because we removed the final inverse rotation, Bob's measurement statistics should reflect |ψ> = Rx(0.7)|0>. Expected probabilities: P(0) ≈ cos^2(0.35) ≈ 0.88, P(1) ≈ sin^2(0.35) ≈ 0.12.

In [ ]:
import math

sim = cirq.Simulator()
result = sim.run(cirq.Circuit(simple_teleportation()), repetitions=1000)
print("Histogram of teleported state measurements (expected ~88% zeros, ~12% ones):")
print(result.histogram(key='result'))

# Expected probabilities
p0 = math.cos(0.35)**2
p1 = math.sin(0.35)**2
print(f"Expected P(0)≈{p0:.3f}, P(1)≈{p1:.3f}")

## 8. Discussion
Custom gates abstract repeated patterns and improve readability, while decomposition supports optimization, hardware mapping, or inspection.